In [30]:
import numpy as np
from scipy.special import softmax

In [31]:
train = np.loadtxt('Assignment1.2/train1.csv', delimiter=',', skiprows=1)
print(train.shape)

(87595, 1184)


In [43]:
# append a column of ones to the train data and remove the last column as class_label
X = train[:, :-1]
one = np.ones((X.shape[0], 1))
X= np.hstack((one,X))
X= X.astype(np.float64)
Y = train[:, -1]
Y = Y.astype(int)-1
print(X.shape, Y.shape)
unique, counts = np.unique(Y, return_counts=True)
print(unique, counts)

(87595, 1184) (87595,)
[0 1 2 3] [16131  1032 25406 45026]


In [75]:
W =np.zeros((X.shape[1], len(unique)), dtype=np.float64)
print(W.shape)

(1184, 4)


In [70]:
def g(W,x,j):
    z = np.dot(x,W)
    sm= softmax(z)
    return sm[j]

def loss(X,Y,W,counts):
    n = X.shape[0]
    Z = X @ W
    softmax_probs = softmax(Z, axis=1)
    indices = (np.arange(n), Y)
    correct_class_probs = softmax_probs[indices]
    scaled_probs = np.log(correct_class_probs) / counts[Y]
    loss_value = -np.mean(scaled_probs) / 2
    return loss_value

def compute_gradient(X, Y, W, counts):
    n, m = X.shape
    k = W.shape[1]
    z = X @ W  
    softmax_probs = softmax(z, axis=1) 
    indices = (np.arange(n), Y)
    Y_one_hot = np.zeros((n, k))
    Y_one_hot[indices] = 1
    grad_W = X.T @ ((softmax_probs - Y_one_hot) / counts[Y][:, np.newaxis]) / (2 * n) 
    return grad_W


In [58]:
def gradient_descent1(X, Y, W, counts, learning_rate, epochs, batch_size):
    n = X.shape[0]
    for i in range(epochs):
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            X_batch = X[start:end]
            Y_batch = Y[start:end]
            batch_loss = loss(X_batch, Y_batch, W, counts)
            print(f"Epoch{i+1}, Batch{1+int(start/batch_size)}, Loss{batch_loss}")

            gradient = compute_gradient(X_batch, Y_batch, W, counts)
            W -= learning_rate * gradient

W = gradient_descent1(X, Y, W, counts, learning_rate=1e-9, epochs=25, batch_size=100)


Epoch1, Batch1, Loss2.537924675372625e-05
Epoch1, Batch2, Loss3.650409771363313e-05
Epoch1, Batch3, Loss2.3152746256393274e-05
Epoch1, Batch4, Loss3.021112783486992e-05
Epoch1, Batch5, Loss2.4051229863262187e-05
Epoch1, Batch6, Loss2.332845781501698e-05
Epoch1, Batch7, Loss2.0810467851515364e-05
Epoch1, Batch8, Loss4.055231202222848e-05
Epoch1, Batch9, Loss3.082145013967208e-05
Epoch1, Batch10, Loss3.737138084046502e-05
Epoch1, Batch11, Loss2.4271279118534135e-05
Epoch1, Batch12, Loss2.3869414762702588e-05
Epoch1, Batch13, Loss2.1841862646183097e-05
Epoch1, Batch14, Loss2.272927498879649e-05
Epoch1, Batch15, Loss3.0177402642666078e-05
Epoch1, Batch16, Loss2.855740915156661e-05
Epoch1, Batch17, Loss2.0854136981992546e-05
Epoch1, Batch18, Loss3.0051702527301494e-05
Epoch1, Batch19, Loss3.2906286299725314e-05
Epoch1, Batch20, Loss3.25407625811694e-05
Epoch1, Batch21, Loss3.027503976657551e-05
Epoch1, Batch22, Loss3.0316008737279444e-05
Epoch1, Batch23, Loss3.0135078166762636e-05
Epoch1, B

In [68]:
def gradient_descent2(X, Y, W, counts, learning_rate, k, epochs, batch_size):
    n = X.shape[0]
    for i in range(epochs):
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            X_batch = X[start:end]
            Y_batch = Y[start:end]
            batch_loss = loss(X_batch, Y_batch, W, counts)
            print(f"Epoch{i+1}, Batch{1+int(start/batch_size)}, Loss{batch_loss}")

            gradient = compute_gradient(X_batch, Y_batch, W, counts)
            W -= learning_rate * gradient /(2+i*k)

W = gradient_descent2(X, Y, W, counts, learning_rate=1e-8, k=1, epochs=25, batch_size=100)


Epoch1, Batch1, Loss2.537924675372625e-05
Epoch1, Batch2, Loss4.034545231410237e-05
Epoch1, Batch3, Loss2.3733496750626156e-05
Epoch1, Batch4, Loss3.19153964214873e-05
Epoch1, Batch5, Loss2.4256521838754652e-05
Epoch1, Batch6, Loss2.173230801235297e-05
Epoch1, Batch7, Loss1.961991249637136e-05
Epoch1, Batch8, Loss4.807775381305885e-05
Epoch1, Batch9, Loss3.065680625136132e-05
Epoch1, Batch10, Loss3.5822015079435574e-05
Epoch1, Batch11, Loss3.14005578864735e-05
Epoch1, Batch12, Loss2.1888456332314707e-05
Epoch1, Batch13, Loss2.0056927494388854e-05
Epoch1, Batch14, Loss2.086854706503506e-05
Epoch1, Batch15, Loss3.52045933576672e-05
Epoch1, Batch16, Loss2.8682956083234732e-05
Epoch1, Batch17, Loss2.0732513367082376e-05
Epoch1, Batch18, Loss3.209972912637208e-05
Epoch1, Batch19, Loss3.200050359151428e-05
Epoch1, Batch20, Loss3.290753614696367e-05
Epoch1, Batch21, Loss2.977875236143579e-05
Epoch1, Batch22, Loss3.099640104567562e-05
Epoch1, Batch23, Loss2.9934473528994517e-05
Epoch1, Batch24

In [76]:
def compute_n(X, Y, W, gradient, n0, counts):
    nl = 0.0
    nh = n0
    while loss(X, Y, W, counts) > loss(X, Y, W - nh*gradient, counts):
        nh *= 2
    for _ in range(20):
        n1 = (2*nl + nh)/3
        n2 = (nl + 2*nh)/3
        if loss(X, Y, W - n1*gradient, counts) > loss(X, Y, W - n2*gradient, counts):
            nl = n1
        else:
            nh = n2
    return (nl+nh)/2

def gradient_descent3(X, Y, W, counts, n0, epochs, batch_size):
    n = X.shape[0]
    for i in range(epochs):
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            X_batch = X[start:end]
            Y_batch = Y[start:end]
            batch_loss = loss(X_batch, Y_batch, W, counts)
            print(f"Epoch{i+1}, Batch{1+int(start/batch_size)}, Loss{batch_loss}")

            gradient = compute_gradient(X_batch, Y_batch, W, counts)
            learning_rate= compute_n(X_batch, Y_batch, W, gradient, n0, counts)
            W -= learning_rate * gradient

W = gradient_descent3(X, Y, W, counts, n0=1e-9, epochs=5, batch_size=100)

Epoch1, Batch1, Loss2.537924675372625e-05
Epoch1, Batch2, Loss4.4749312456964797e-05
Epoch1, Batch3, Loss2.3119251881876762e-05
Epoch1, Batch4, Loss4.265245176692738e-05
Epoch1, Batch5, Loss2.5132776938601487e-05
Epoch1, Batch6, Loss2.0929811988510988e-05
Epoch1, Batch7, Loss1.900685238247395e-05
Epoch1, Batch8, Loss5.4436497469976615e-05
Epoch1, Batch9, Loss3.117491527645202e-05
Epoch1, Batch10, Loss3.5668315595225346e-05
Epoch1, Batch11, Loss2.8375043501995058e-05
Epoch1, Batch12, Loss2.028806864880217e-05
Epoch1, Batch13, Loss1.8933986627881282e-05
Epoch1, Batch14, Loss2.0010041106125424e-05
Epoch1, Batch15, Loss4.549227837155885e-05
Epoch1, Batch16, Loss2.902866925607281e-05
Epoch1, Batch17, Loss2.1444915343205197e-05
Epoch1, Batch18, Loss3.839960929093427e-05
Epoch1, Batch19, Loss3.2513163946197945e-05
Epoch1, Batch20, Loss3.307424717519658e-05
Epoch1, Batch21, Loss2.9934346365310993e-05
Epoch1, Batch22, Loss3.0899070325665725e-05
Epoch1, Batch23, Loss2.993348987013503e-05
Epoch1,